<a href="https://colab.research.google.com/github/ensw1031/class-codes/blob/master/webscrapy_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imports
from bs4 import BeautifulSoup

import requests

In [6]:
#variaveis importantes

base_url = 'https://www.raulfulgencio.com.br'

apts = []

In [65]:
# função para extrair as informações do apt

def extract_apt_info(apt):
  price = apt.find('div', class_='price').find('span').text.strip()
  title = apt.find('div', class_='info').find('h3').find('a').text.strip()
  neighborhood = apt.find('div', class_='info').find('h3').find('small').text.strip()
  # print(f'{title} ({neighborhood}) {price}')
  # area = apt.find('i', class_='icon-area').parent.text.split(' ')[0]
  # bedrooms = apt.find('i', class_='icon-bedrooms')  
  # bathrooms = apt.find('i', class_='icon-bathrooms').parent.text.strio()
  # print(f'{title} ({neighborhood}) {price} - {area} ({bedrooms}, bathrooms)')
  detail_url = apt.find('a').get('href')
  # print(base_url+detail_url)
  print(f"GET {base_url}/{detail_url}")
  page = requests.get(f"{base_url}/{detail_url}")
  soup = BeautifulSoup(page.content, 'html.parser')
  amenities = soup.find_all('ul', class_='property-amenities-list col-md-6')
  apt = {
      'price': price,
      'title': title,
      'neighborhood': neighborhood,
  }

  info_dict = dict()
  for col in amenities:
    infos = col.find_all('li')
    for info in infos:
      tokens = info.text.split(':')      
      key = tokens[0].strip().lower()
      value = tokens[1].strip()
      info_dict[key] = value
      # print(f"{key}: {value}")

  if('suítes' not in info_dict):
    info_dict['suítes'] = 0
    
  if('iptu' not in info_dict):
    info_dict['iptu'] = 0

  if('vagas na garagem' not in info_dict):
    info_dict['vagas na garagem'] = 0

  apt['info'] = info_dict

  return apt

In [66]:
x = range(1, 16)
for n in x: 
  url = f"{base_url}/alugar/Londrina/Apartamento/Padrao/?pag={n}"
  print(f"GET {url}")
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  apes = soup.find_all('div', class_='item col-sm-6 col-md-4 col-lg-3')
  for ape in apes:
    price = ape.find('div', class_='price').find('span').text.strip()
    if('Consulte' not in price):    
      extract_apt = extract_apt_info(ape)
      # print(extract_apt)
      apts.append(extract_apt)



GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/?pag=1
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Jamaica/6657
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Centro/6649
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Jardim-Higienopolis/6645
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/San-Remo/6619
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Gleba-Fazenda-Palhano/6594
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Jardim-Agari/6587
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Centro/6573
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Gleba-Fazenda-Palhano/6571
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Gleba-Fazenda-Palhano/6525
GET https://www.raulfulgencio.com.br/alugar/Londrina/Apartamento/Padrao/Vila-Larsen-1/6501
GET htt

In [67]:
len(apts)

295

### Salvar os dados raw usando o pickle

In [10]:
import pickle

In [68]:
with open('/content/drive/My Drive/Classroom/ENSW1031- Big Data ENSW306K/data/aptos.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(apts, f, pickle.HIGHEST_PROTOCOL)

### Abrindo o arquivo salvo

In [95]:
with open('/content/drive/My Drive/Classroom/ENSW1031- Big Data ENSW306K/data/aptos.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data = pickle.load(f)

In [92]:
def clean_info(info):
  new_info = dict()
  if not isinstance(info['dormitórios'], int):
    new_info['dormitórios'] = int(info['dormitórios'].split(' ')[0])
  new_info['banheiros'] = int(info['banheiros'])
  new_info['suítes'] = int(info['suítes'])
  if 'vagas na garagem' in info:
    new_info['vagas na garagem'] = int(info['vagas na garagem'])
  if not isinstance(info['área útil'], float):
    new_info['área útil'] = float(info['área útil'].split(' ')[0].replace(',','.'))
  if not isinstance(info['condomínio'], float):
    new_info['condomínio'] = float(clean_price(info['condomínio'].split(' ')[0]))
  if not isinstance(info['iptu'], int) and not isinstance(info['iptu'], float):
    new_info['iptu'] = float(clean_price(info['iptu'].split(' ')[0]))
  
  return new_info

In [99]:
def clean_price(price):
  if not isinstance(price, float):
    return price.replace('.','').replace(',','.')
  return price

In [100]:
for apt in apts:
  print(apt)
  apt['price'] = float(clean_price(apt['price']))
  apt['new_info'] = clean_info(apt['info'])


{'price': 1950.0, 'title': 'Apartamento Padrão', 'neighborhood': 'Jamaica', 'info': {'operação': 'Locação', 'cidade': 'Londrina', 'dormitórios': '2', 'banheiros': '1', 'vagas na garagem': '1', 'tipo do imóvel': 'Padrão', 'bairro': 'Jamaica', 'condomínio': '230,00 / Mês - (Sujeito a alterações)', 'área útil': '47.00 M²', 'suítes': 0, 'iptu': 0}, 'new_info': {'dormitórios': 2, 'banheiros': 1, 'suítes': 0, 'vagas na garagem': 1, 'área útil': 47.0, 'condomínio': 230.0}}
{'price': 1950.0, 'title': 'Apartamento Padrão', 'neighborhood': 'Centro', 'info': {'operação': 'Locação', 'cidade': 'Londrina', 'dormitórios': '3 \n                                \t\t\t\t\t\t\t\t\t\t\t/ Sendo 1 Suite', 'banheiros': '3', 'vagas na garagem': '2', 'tipo do imóvel': 'Padrão', 'bairro': 'Centro', 'condomínio': '902,00 / Mês - (Sujeito a alterações)', 'suítes': '1', 'área útil': '137.09 M²', 'iptu': 0}, 'new_info': {'dormitórios': 3, 'banheiros': 3, 'suítes': 1, 'vagas na garagem': 2, 'área útil': 137.09, 'cond

In [63]:
dict_test = {
    'price': 1950.0, 
    'title': 'Apartamento Padrão', 
    'neighborhood': 'Jardim Higienópolis', 
    'info': {
        'operação': 'Locação', 
        'cidade': 'Londrina', 
        'dormitórios': '2', 
        'banheiros': '1', 
        'tipo do imóvel': 
        'Padrão', 
        'bairro': 'Jardim Higienópolis', 
        'condomínio': '280,00 / Mês - (Sujeito a alterações)', 
        'área útil': '50.00 M²', 
        'suítes': 0, 
        'iptu': 0
        }
 }


In [102]:
apts[0]

{'info': {'bairro': 'Jamaica',
  'banheiros': '1',
  'cidade': 'Londrina',
  'condomínio': '230,00 / Mês - (Sujeito a alterações)',
  'dormitórios': '2',
  'iptu': 0,
  'operação': 'Locação',
  'suítes': 0,
  'tipo do imóvel': 'Padrão',
  'vagas na garagem': '1',
  'área útil': '47.00 M²'},
 'neighborhood': 'Jamaica',
 'new_info': {'banheiros': 1,
  'condomínio': 230.0,
  'dormitórios': 2,
  'suítes': 0,
  'vagas na garagem': 1,
  'área útil': 47.0},
 'price': 1950.0,
 'title': 'Apartamento Padrão'}